In [104]:
import sys
sys.path.append('../../')
import os

import pandas as pd
from utils import fill_empty, VColumns
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load input file

In [105]:
df1 = pd.read_csv("v1-start.csv")

# Start elaborating columns

Create a new dataframe with empty values, so that we can start filling in values from the input dataframe, without overriding columns.

In [106]:
df1.head()

,vintage,name,size,type,last_supplier,last_purchase_date,last_purchase_qty,fallback_purchase_price,qty,price_eur
0,NaN,Arunda Brut - Half bottle,NaN,NaN,Weindiele,06/12/2023,6.0,"9,30 €",7.0,NaN
1,NaN,Arunda Blanc de Blanc,NaN,NaN,Vinum Bozen,06/12/2023,6.0,"21,00 €",10.0,NaN
2,NaN,Bellavista Brut Alma,NaN,NaN,Vinum Bozen,06/03/2025,4.0,"29,00 €",8.0,NaN
3,2014.0,Brut Balter Trentodoc,NaN,NaN,Liber International,06/06/2023,6.0,"24,40 €",8.0,NaN
4,NaN,Bollé Rosè Spumante,NaN,NaN,Agricola Andreola,16/12/2023,30.0,"5,40 €",27.0,NaN


In [107]:
# create new empty dataframe
df = pd.DataFrame(columns=VColumns.v2())
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible


In [108]:
import re

# copy values from original columns to new columns
df['external_id'] = None
df['type'] = df1['type']
df['name'] = df1['name']
df['winery_name'] = None
df["vintage"] = df1["vintage"]
df["quantity"] = df1["qty"]
df["storage_area"] = None
df["visible"] = True

# store last purchase in internal notes
df1["last_purchase_qty"] = df1["last_purchase_qty"].fillna("0")
df["internal_notes"] = df1.apply(lambda x: f"Ultimo acquisto pre-vinoteqa  ({x['last_purchase_date']}): {int(x['last_purchase_qty'])} x {x['fallback_purchase_price']} da {x['last_supplier']}", axis=1)

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,NaN,NaN,Arunda Brut - Half bottle,None,NaN,NaN,NaN,NaN,NaN,7.0,None,Ultimo acquisto pre-vinoteqa (06/12/2023): 6 ...,True
1,NaN,NaN,Arunda Blanc de Blanc,None,NaN,NaN,NaN,NaN,NaN,10.0,None,Ultimo acquisto pre-vinoteqa (06/12/2023): 6 ...,True
2,NaN,NaN,Bellavista Brut Alma,None,NaN,NaN,NaN,NaN,NaN,8.0,None,Ultimo acquisto pre-vinoteqa (06/03/2025): 4 ...,True
3,NaN,NaN,Brut Balter Trentodoc,None,NaN,NaN,2014.0,NaN,NaN,8.0,None,Ultimo acquisto pre-vinoteqa (06/06/2023): 6 ...,True
4,NaN,NaN,Bollé Rosè Spumante,None,NaN,NaN,NaN,NaN,NaN,27.0,None,Ultimo acquisto pre-vinoteqa (16/12/2023): 30...,True


In [109]:
# size
df['size'] = df1["size"].fillna("BOTTLE")

# fill empty prices with 0
df1['fallback_purchase_price'] = df1['fallback_purchase_price'].fillna("0,00")
df1['price_eur'] = df1['price_eur'].fillna(0)

# convert prices to cents
df['purchase_price'] = df1['fallback_purchase_price'].apply(lambda x: int(float(str(x).replace('€', '').replace(',', '.'))*100)) # convert to cents
df['price'] = df1['price_eur'].apply(lambda x: int(float(str(x).replace('€', '').replace(',', '.'))*100)) # convert to cents

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,NaN,NaN,Arunda Brut - Half bottle,None,NaN,BOTTLE,NaN,0,930,7.0,None,Ultimo acquisto pre-vinoteqa (06/12/2023): 6 ...,True
1,NaN,NaN,Arunda Blanc de Blanc,None,NaN,BOTTLE,NaN,0,2100,10.0,None,Ultimo acquisto pre-vinoteqa (06/12/2023): 6 ...,True
2,NaN,NaN,Bellavista Brut Alma,None,NaN,BOTTLE,NaN,0,2900,8.0,None,Ultimo acquisto pre-vinoteqa (06/03/2025): 4 ...,True
3,NaN,NaN,Brut Balter Trentodoc,None,NaN,BOTTLE,2014.0,0,2440,8.0,None,Ultimo acquisto pre-vinoteqa (06/06/2023): 6 ...,True
4,NaN,NaN,Bollé Rosè Spumante,None,NaN,BOTTLE,NaN,0,540,27.0,None,Ultimo acquisto pre-vinoteqa (16/12/2023): 30...,True


In [110]:
# discard rows with 0 qty
df = df[df["quantity"] > 0]

df = fill_empty(df, VColumns.v2(), False)
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,,,Arunda Brut - Half bottle,,,BOTTLE,0,0,930,7,,Ultimo acquisto pre-vinoteqa (06/12/2023): 6 ...,True
1,,,Arunda Blanc de Blanc,,,BOTTLE,0,0,2100,10,,Ultimo acquisto pre-vinoteqa (06/12/2023): 6 ...,True
2,,,Bellavista Brut Alma,,,BOTTLE,0,0,2900,8,,Ultimo acquisto pre-vinoteqa (06/03/2025): 4 ...,True
3,,,Brut Balter Trentodoc,,,BOTTLE,2014,0,2440,8,,Ultimo acquisto pre-vinoteqa (06/06/2023): 6 ...,True
4,,,Bollé Rosè Spumante,,,BOTTLE,0,0,540,27,,Ultimo acquisto pre-vinoteqa (16/12/2023): 30...,True


# Merge files and write output

If there are multiple files, merge them into one and create single output file.

In [112]:
df.head()

df_out = pd.concat([df], ignore_index=True)
df_out.to_csv("v2-cleaned.csv", index=False)

bool()

False